In [19]:
import psycopg2

In [20]:
""" Connect to database """
conn_params = "dbname='trajectory' user='gpadmin' host='192.168.59.101' password='1234' port=5432"
conn = psycopg2.connect(conn_params)
cur = conn.cursor()

In [21]:
sql_function_extension = """
CREATE extension plpythonu;
Create extension postgis;
"""
cur.execute(sql_function_extension)

In [22]:
sql_function_aggregation = """

CREATE OR REPLACE FUNCTION aggregation(indexing_key INTEGER, depth INTEGER)
                RETURNS INTEGER[]
            AS
            $$
                m_b = bin(indexing_key)[2:].zfill(depth * 3)
                x_b, y_b, t_b = "", "", ""
                for i in range(depth):
                    x_b += m_b[3 * i]
                    y_b += m_b[3 * i + 1]
                    t_b += m_b[3 * i + 2]
                return [int(x_b, 2), int(y_b, 2), int(t_b, 2)]
            $$ LANGUAGE plpythonu; """
cur.execute(sql_function_aggregation)

In [23]:
sql_function_projection = """ CREATE OR REPLACE FUNCTION projection(indexing_key INTEGER, depth INTEGER)
                RETURNS INTEGER[]
            AS
            $$
                m_b = bin(indexing_key)[2:].zfill(depth * 3)
                x_b, y_b = "", ""
                for i in range(depth):
                    x_b += m_b[3 * i]
                    y_b += m_b[3 * i + 1]
                return [int(x_b, 2), int(y_b, 2)]
            $$ LANGUAGE plpythonu; """
cur.execute(sql_function_projection)

In [24]:
conn.commit()
cur.close()
conn.close()